In [1]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models.google_palm import ChatGooglePalm

In [2]:
import os
api_key = os.getenv("PALM_API_KEY")

In [3]:

# Initialize ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "Hi"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"}, {"output": "Cool"})

# Load memory variables
memory.load_memory_variables({})


{'history': 'Human: Not much, just hanging\nAI: Cool'}

### Candidate Count Errors
There is a potential error that langchain could raise when you ask for a prediction. This is `ChatGooglePalmError`. The reason this is happening is that when temperature is set to low values, the llm returns `None`. Hence why it says: **ChatResponse must have at least one candidate**. 

> candidate_count: The **maximum** number of generated response messages to return.
    
            This value must be between `[1, 8]`, inclusive. If unset, this
            will default to `1`.
    
            Note: Only unique candidates are returned. Higher temperatures are more
            likely to produce unique candidates. Setting `temperature=0.0` will always
            return 1 candidate regardless of the `candidate_count`.

The patch for now is to set temperature high.

Also setting k to high seems to also mitigate this error.

In [28]:
llm = ChatGooglePalm(google_api_key=api_key, temperature=0.7)

In [32]:
# Create ConversationChain
memory = ConversationBufferWindowMemory(k=5)
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

# Predict using the ConversationChain
conversation.predict(input="Hi, my name is Andrew")

# Additional predictions
conversation.predict(input="What is 1+1?")
conversation.predict(input="What is my name?")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hi Andrew, how can I help you today?
Human: What is 1+1?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific d

"Hello Andrew, it's nice to meet you!\r\n\r\nYour name is Andrew."